<a href="https://colab.research.google.com/github/chunabayashi0216/new_work/blob/main/tounyou_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install japanize-matplotlib

     |████████████████████████████████| 4.1MB 5.9MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-cp37-none-any.whl size=4120276 sha256=2c07de0e6e3fff76175587f85193fc8473144d274f95f056bb0a5dd5da72a49c
  Stored in directory: /root/.cache/pip/wheels/b7/d9/a2/f907d50b32a2d2008ce5d691d30fb6569c2c93eefcfde55202
Successfully built japanize-matplotlib


In [2]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

In [18]:
# Load in the train and test datasets
train_raw = pd.read_csv('/content/train.csv') #元の学習データ
test_raw = pd.read_csv('/content/test.csv') #元のテストデータ

In [19]:
print('The size of the df_train data:' + str(train_raw.shape))
print('The size of the df_test data:' + str(test_raw.shape))


The size of the df_train data:(3000, 10)
The size of the df_test data:(2000, 9)


In [20]:
df_train = train_raw.copy() #trainデータのコピー
df_test = test_raw.copy() #testデータのコピー

In [21]:
df_train['train_or_test'] = 'train' #trainデータの識別カラム追加 
df_test['train_or_test'] = 'test' #testデータの 識別カラム追加  
df_test['Y'] = 9 #testデータに'Y'を仮置き

In [22]:
df_all = pd.concat([df_train,df_test],sort=False,axis=0).reset_index(drop=True)

In [23]:
df_all.head()

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,train_or_test,Y
0,200,9,125,74,0,0,28.536910,0.444902,45,1.0,train,NaN
1,3832,4,109,80,0,0,28.047673,0.238243,22,0.0,train,NaN
2,4927,4,88,78,39,0,52.371341,0.279471,26,0.0,train,NaN
3,4088,9,125,74,0,0,40.062688,0.203922,45,0.0,train,NaN
4,3644,5,107,78,44,284,52.935068,0.284959,45,1.0,train,NaN



*   0	index				BMIint	インデックスとして使用
*   1	Pregnancies			int	妊娠した回数
*   2	Glucose				int	経口ブドウ糖負荷試験における2時間の血漿ブドウ糖濃度
*   3	BloodPressure			int	拡張期血圧
*   4	SkinThickness			int	皮膚のひだの厚さ
*   5	Insulin				int	血清インスリン
*   6	BMI				float	BMI
*   7	Age				int	年齢
*   8	DiabetesPedigreeFunction	float	糖尿病血統
*   9	Outcome				int	糖尿病であるか（糖尿病の場合1、でない場合0）

In [24]:
df_all.describe() # df_fullの要約統計量

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Y
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,3000.000000,2000.0
mean,2499.500000,3.567800,113.957600,68.752800,11.180000,11.741600,35.347125,0.400987,28.989400,0.239000,9.0
std,1443.520003,3.040535,21.333505,16.268147,14.232446,47.022061,7.496245,0.271619,8.509722,0.426544,0.0
min,0.000000,0.000000,57.000000,0.000000,0.000000,0.000000,0.000775,0.137377,21.000000,0.000000,9.0
25%,1249.750000,1.000000,100.000000,64.000000,0.000000,0.000000,32.453978,0.232290,22.000000,0.000000,9.0
50%,2499.500000,3.000000,111.000000,70.000000,0.000000,0.000000,33.828671,0.269691,26.000000,0.000000,9.0
75%,3749.250000,6.000000,125.000000,78.000000,24.000000,0.000000,39.627290,0.506604,33.000000,0.000000,9.0
max,4999.000000,13.000000,199.000000,110.000000,52.000000,744.000000,53.400629,2.302072,67.000000,1.000000,9.0


In [27]:
# 前処理
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   index                     5000 non-null   int64  
 1   Pregnancies               5000 non-null   int64  
 2   Glucose                   5000 non-null   int64  
 3   BloodPressure             5000 non-null   int64  
 4   SkinThickness             5000 non-null   int64  
 5   Insulin                   5000 non-null   int64  
 6   BMI                       5000 non-null   float64
 7   DiabetesPedigreeFunction  5000 non-null   float64
 8   Age                       5000 non-null   int64  
 9   Outcome                   3000 non-null   float64
 10  train_or_test             5000 non-null   object 
 11  Y                         2000 non-null   float64
dtypes: float64(4), int64(7), object(1)
memory usage: 468.9+ KB


In [28]:
# 欠損値なし　確認
df_all.isnull().sum()

index                          0
Pregnancies                    0
Glucose                        0
BloodPressure                  0
SkinThickness                  0
Insulin                        0
BMI                            0
DiabetesPedigreeFunction       0
Age                            0
Outcome                     2000
train_or_test                  0
Y                           3000
dtype: int64

モデル構築

In [25]:
from sklearn.model_selection import train_test_split

# 最初に統合したtrainとtestを分離
train = df_all.query('train_or_test == "train"')
test = df_all.query('train_or_test == "test"')

# 学習に必要な特徴量のみを保持
target_col = 'Y'
drop_col = 'Y'

train_feature = train[drop_col]
test_feature = test[drop_col]
train_tagert = train[target_col]

In [32]:
df_all.head(5)

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,train_or_test,Y
0,200,9,125,74,0,0,28.536910,0.444902,45,1.0,train,NaN
1,3832,4,109,80,0,0,28.047673,0.238243,22,0.0,train,NaN
2,4927,4,88,78,39,0,52.371341,0.279471,26,0.0,train,NaN
3,4088,9,125,74,0,0,40.062688,0.203922,45,0.0,train,NaN
4,3644,5,107,78,44,284,52.935068,0.284959,45,1.0,train,NaN


In [26]:
# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    train_feature, train_tagert, test_size=0.2, random_state=0, stratify=train_tagert )

ValueError: ignored

In [17]:
print('train_feature = ', train_feature.shape)
print('test_feature = ', test_feature.shape)
print('train_tagert = ', train_tagert.shape)
print('X_train = ', X_train.shape)
print('X_test = ', X_test.shape)
print('y_train = ', y_train.shape)
print('y_test = ', y_test.shape)

train_feature =  (3000,)
test_feature =  (2000,)
train_tagert =  (3000,)


NameError: ignored